This tutorial/handson session is designed based on the Visual Turing Test tutorial in here:
https://github.com/mateuszmalinowski/visual_turing_test-tutorial
Visual Turing Challenge
Mateusz Malinowski and Mario Fritz
Max-Plank Institute

Mehdi Ghanimifard

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

import numpy as np
#from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image as kimage

Using TensorFlow backend.


In [2]:
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Embedding, Concatenate, Dropout
from keras.layers import Input
from keras.callbacks import EarlyStopping
import keras.backend as K


In [3]:
import glob

In [53]:
from random import shuffle

The following cell is to show how Mehdi's code from the tutorial would work. In my case, my image model predicts classes (angry/happy), and not features. My bottleneck_fc_model.h5 predicts features, but these are actually already loaded in bottleneck_features_train and validation.npy (from run of eslp_image.py), which I load in the next cell.

In [24]:
# Loads my image_model (the one that predicts features, not classes) 

#pretrained_cnn_model = load_model('2bottleneck_fc_model.h5')
#pretrained_cnn_model = load_model('2bottleneck_fc_model.h5')

# a function (from image file path to feature vectors)
# From Mehdi's tutorial

#def img2vec(image_path):
#    x = kimage.load_img(image_path, target_size=[48,48])#from 224,224
#    x_array = kimage.img_to_array(x)
#    xs_array = np.array([x_array,])
#    # notice that we are not using full capacity of the GPU when we are passing only one image per prediction.
#    # we could have a larger batch.
#    return pretrained_cnn_model.predict(preprocess_input(xs_array)).flatten()

#use model to extract features from .png format pictures with our function img2vec: 

#angry_images=[]
#happy_images=[]

#path = "pngs/happy/output*.png"
#for png in glob.glob(path):
#    happy_images.append(img2vec(png))
#print('Happy faces:   ', len(happy_images))  

#path = "pngs/angry/output*.png"
#for png in glob.glob(path):
#    angry_images.append(img2vec(png))
#print('Angry faces:   ', len(angry_images))

In [41]:
b_train=np.load('bottleneck_features_train.npy')
b_val=np.load('bottleneck_features_validation.npy')

In [37]:
b_train.shape

(7904, 1, 1, 512)

In [42]:
b_val.shape

(1952, 1, 1, 512)

In [142]:
#these features are half angry, half happy, so I save them accordingly:
angry_images_train=b_train[:int((7904/2))]
angry_images_test=b_val[:int((1952/2))]
happy_images_train=b_train[int((7904/2)):]
happy_images_test=b_val[int((1952/2)):]

In [48]:
#Load sentences (from run of preprocessing.py): 
happy_sents=np.load('happy_sents.npy')#With sentiment score above 0.6
angry_sents=np.load('angry_sents.npy')#With sentiment score below 0.4

In [49]:
#Splits each category and input in train and test before mapping. 
#When mapping, one image is paired with some sentences, and we dont want to shuffle some anwers into the test set.

#train_angry_images=angry_images[:int(0.8*len(angry_images))]
#test_angry_images=angry_images[int(0.8*len(angry_images)):]

#train_happy_images=happy_images[:int(0.8*len(happy_images))]
#test_happy_images=happy_images[int(0.8*len(happy_images)):]

train_angry_sents=angry_sents[:int(0.8*len(angry_sents))]
test_angry_sents=angry_sents[int(0.8*len(angry_sents)):]

train_happy_sents=happy_sents[:int(0.8*len(happy_sents))]
test_happy_sents=happy_sents[int(0.8*len(happy_sents)):]

In [ ]:
#np.save("angry_images.npy", angry_images)
#np.save("happy_images.npy", happy_images)

In [ ]:
#angry_images = [
#    img2vec("pngs/angry/{0}.png".format(image_name.strip()))
#    for image_name in open('pngs/angry/{0}.png')
#]

In [ ]:
# this code takes time:
#X_images = [
#    img2vec("data/daquar/images/{0}.png".format(image_name.strip()))
#    for image_name in open('data/daquar/qa.894.raw.train.format_triple.contexts')
#]

In [104]:
len(angry_images_train[0][0][0])

512

In [112]:
#Maps each image with with the chosen sentences. 
#Every image is mapped with more than one sentence, to grow the data size

#Angry face with negative sentences
#happy face with positive sentences
#Saves the sentiment category as 'NEG' or 'POS'


train=[]
test=[]

sent_count=0
for image in angry_images_train:    
    for step in range(6):
        train.append((image[0][0], train_angry_sents[sent_count][0], 'NEG'))
        sent_count=+1
sent_count=0
for image in happy_images_train:    
    for step in range(6):
        train.append((image[0][0], train_happy_sents[sent_count][0], 'POS'))
        sent_count=+1
        
sent_count=0        
for image in happy_images_test:
    for step in range(5):
        test.append((image[0][0], test_happy_sents[sent_count][0], 'POS'))
        sent_count=+1 
sent_count=0        

for image in angry_images_test:
    for step in range(5):
        test.append((image[0][0], test_angry_sents[sent_count][0], 'NEG'))
        sent_count=+1 
    
print('total pairs: ',len(train)+len(test))

total pairs:  57184


In [113]:
#Now we can shuffle in each set:
shuffle(train)
shuffle(test)

In [57]:
#s=int((len(all_data)*0.8))#splits with 80% training, 20% test
#train, test=all_data[:s], all_data[s:]

In [114]:
xtrain=[(image,sent) for (image,sent,label) in train]
ytrain=[label for (image,sent,label) in train]
xtest=[(image,sent) for (image,sent,label) in test]
ytest=[label for (image,sent,label) in test]

#image_size=len(train[0][0])
#sent_size=len(train[0][1])
#labels=set(list(l for (i,s,l) in all_data))

xtrain2 = list(zip(*xtrain))
xtrain2 = [np.array(xtrain2[1]), np.array(xtrain2[0])]

xtest2 = list(zip(*xtest))
xtest2 = [np.array(xtest2[1]), np.array(xtest2[0])]

In [153]:
set(list(l for l in ytrain))

{'NEG', 'POS'}

In [154]:
cat_codes = {
    'POS': 0,
    'NEG': 1
}

In [155]:
ytrain2 = np.array([cat_codes[c] for c in ytrain])
ytest2 = np.array([cat_codes[c] for c in ytest])

In [156]:
np.save("xtrain0.npy", xtrain2[0])#image
np.save("xtrain1.npy", xtrain2[1])#sent
np.save("ytrain.npy", ytrain2)#label
np.save("xtest0.npy", xtest2[0])#image
np.save("xtest1.npy", xtest2[1])#sent
np.save("ytest.npy", ytest2)#label

In [157]:
image_size=len(xtrain[0][0])
sent_size=len(xtrain[0][1])
vocab = list(np.load('vocab.npy'))
labels = {'POS', 'NEG'}

In [140]:
#xtrain = [np.load('xtrain0.npy'), np.load('xtrain1.npy')]
#ytrain = np.load('ytrain.npy')
#xtest = [np.load('xtest0.npy'), np.load('xtest1.npy')]
#ytest = np.load('ytest.npy')



In [130]:
input_question = Input([sent_size,])
input_context = Input([image_size,])

# learn embedings (size=50 as we chose just now :D)
q_embs = Embedding(len(vocab), 50)(input_question)

# encode the question
q_encoded = LSTM(50)(q_embs)

mlp_1 = Dense(image_size, activation='tanh')(q_encoded)

q_composed = Concatenate()([input_context, mlp_1])

mlp_2 = Dropout(0.2)(Dense(image_size, activation='relu')(q_composed))
#mlp_2 = Dropout(0.2)(Dense(image_size, activation='relu')(mlp_1))

final_a = Dense(len(labels), activation='softmax')(mlp_2)

model = Model([input_question, input_context], final_a)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 56)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 56, 50)       905600      input_7[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 50)           20200       embedding_4[0][0]                
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
dense_10 (

In [133]:

model.compile('adam', 'sparse_categorical_crossentropy', ['accuracy'])

In [ ]:
history=model.fit(xtrain2, ytrain2, epochs=50, batch_size=64, validation_split=0.1, callbacks=[EarlyStopping(patience=5)])

In [135]:
model.save('combi_model.h5')  # creates a HDF5 file 'my_model.h5'


In [ ]:
print(history.history)
modelfile = open('./combihistory.txt',"w")
modelfile.write(str(history.history))
modelfile.close()

In [ ]:
### summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('combiloss.pdf')

In [ ]:
### summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('combiaccuracy.pdf')

In [ ]:
#model.evaluate(xtest2, ytest)

In [ ]:
#predictions = model.predict([xtest[0][:1], xtest[1][:1]])
#print('answer predictions', predictions)